# Fantasy Football Analysis
Utilizing the yahoo-fantasy-api, this notebook explores data from the fantasy football league to draw analytical conclusions.

Start by pulling in the necessary libraries and generating OAuth2 configuration from Yahoo's API.

In [ ]:
from yahoo_oauth import OAuth2
import yahoo_fantasy_api as yfa
import yff
import google_sheets as gs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import re
%matplotlib inline

from importlib import reload
reload(gs)
reload(yff)

oauth = OAuth2(None, None, from_file='yahoo.json')

## Explore 2020 Season

In [ ]:
reload(yff)

# Create season object
ssn20 = yff.fantasy_season(oauth, 2020)

In [ ]:
ssn20.get_team_data('9')

## Draft Prep
Pull Player Data and Rankings From Various Websites

In [73]:
# Sporting News
def sn_df():
    url = 'https://www.sportingnews.com/us/fantasy/news/fantasy-superflex-rankings-2021-top-200-ppr/ze1ic3xfl0gd1eptktn9jkodm'
    df = pd.read_html(url, header=0)[0]
    split_df = df['Player'].str.split(',', expand=True)
    df['Player'] = split_df[0]
    df['Team'] = split_df[1]
    return df

# ESPN
def espn_df():
    url = 'https://www.espn.com/fantasy/football/story/_/id/32008027/tristan-h-cockcroft-updated-2021-fantasy-football-rankings-superflex-2-qb-leagues'
    req = requests.get(url)
    soup = BeautifulSoup(req.content)
    para = ["Player", "Eligible Pos", "Team", "Pos Rank"]
    ls = soup.find_all('p')[2].text
    ls = re.split(r'\n', ls)  # remove newline characters
    df = pd.DataFrame([re.split(r'(?:\A\w+\.)|(?:,)', txt) for txt in ls]).drop(0, axis=1).drop(0, axis=0)
    df.columns = para
    df = df.rename({'Eligible Pos': 'Position'}, axis=1)
    return df

# CBS Sports
def cbs_df():
    df = pd.read_csv(r"C:\Users\jborman\Downloads\cbs.csv", header=None)
    df['Position'] = df[0].str.extract(r'\((\w+)\)')
    name_team = df[0].str.slice(stop=-4).str.rstrip()
    df['Team'] = name_team.str.extract(r'(\w+\Z)')
    df['Player'] = name_team.str.split(r'\xa0').str[0]
    df = df[['Player', 'Team', 'Position']]
    return df

In [74]:
# _ = [print(df.Team.unique()) for df in [sn(), espn(), cbs()]]

sn = sn_df()
sn = sn.rename({'Rank': 'SN Rank'}, axis=1)
sn


,Rank,Player,Position,Team
0,1,Patrick Mahomes,QB,Chiefs
1,2,Christian McCaffrey,RB,Panthers
2,3,Kyler Murray,QB,Cardinals
3,4,Josh Allen,QB,Bills
4,5,Alvin Kamara,RB,Saints
...,...,...,...,...
195,196,Cordarrelle Patterson,RB,Falcons
196,197,Kadarius Toney,WR,Giants
197,198,Elijah Moore,WR,Jets
198,199,Zach Ertz,TE,Eagles
